In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [5]:
class MySequential(nn.Module):
    def __init__(self,*layers):
        super(MySequential, self).__init__()

        for i,layer in enumerate(layers):
            self._modules[str(i)]=layer
    def forward(self, input):
        for block in self._modules.values():
            input=block(input)
        return input

在正向传播函数中执行代码
正向传播中随便改 反向传播自动保留计算图 直接backward就行

In [6]:

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super(FixedHiddenMLP, self).__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=True)
        self.linear = nn.Linear(20,20)
    def forward(self, x):
        x=self.linear(x)
        x=F.relu(torch.mm(x,self.rand_weight)+1)
        x=self.linear(x)
        while x.abs().sum()>0.00001:
            x/=2
        return x.sum()

In [7]:
f=FixedHiddenMLP()


In [9]:
f(x)

tensor(2.3689e-06, grad_fn=<SumBackward0>)

可以混合sequential 和任何模型 达到自己想要的模型

In [10]:
class NestMlp(nn.Module):
    def __init__(self):
        super(NestMlp, self).__init__()
        self.net=nn.Sequential(nn.Linear(20,20),nn.ReLU(),nn.Linear(20,20),nn.ReLU(),nn.Linear(20,20))
        self.Linear = nn.Linear(20,20)
    def forward(self, x):
        return self.Linear(self.net(x))

In [11]:
chimear=nn.Sequential(NestMlp(),nn.Linear(20,20),nn.ReLU(),nn.Linear(20,20),nn.ReLU(),nn.Linear(20,1))

In [14]:
x=torch.randn(1,20)
print(chimear(x))

tensor([[0.0097]], grad_fn=<AddmmBackward0>)


In [ ]:
x